<a href="https://colab.research.google.com/github/liuxx479/stickgame/blob/main/stick_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy import *
import numpy as np
import copy

In [2]:
b=np.arange(1,4)
print (b)
print (np.delete(b,b==3))
np.concatenate([b,np.array([3,4])])

[1 2 3]
[1 2]


array([1, 2, 3, 3, 4])

In [3]:
def init_board (lv):
    '''initialize the board'''
    out=np.arange(lv)+1
    return out

print ('test init_board(4):', init_board(4))

test init_board(4): [1 2 3 4]


In [48]:
def split_num (num): ## e.g. 5
    '''split each row to possible new outcomes'''
    # out=[]
    out=[[],] ## can always entirely cross the whole row, so start with a []
    if num==1:
        return out
    for inum in np.arange(num):  ## initial idx
        n0=inum
        for jnum in np.arange(inum+1,num): ## final idx
            n1=num-jnum
            if n1<n0: ## [2,1] and [1,2] are the same, stop computing
                break
            if n0!=0:
                out.append([n0,n1])
            else:
                out.append([n1])
    return out

print ('test split 5:', split_num(5))

test split 5: [[], [4], [3], [2], [1], [1, 3], [1, 2], [1, 1], [2, 2]]


In [147]:
# test code
# board=np.concatenate(split_num(5))
# print (board)
# print ((board==1)%2)

def clean_board (board):
    '''remve double 1, and sort numbers'''
#     print ('sum(board==1)',sum(board==1))
    board=np.array(board)

    odd1s=sum(board==1)%2 ## True if odd number of 1s
#     print (odd1s)
    board=board[board!=1]
    if odd1s:
        board=np.append(board,[1])

    ## second, sort the array, so only split the ones that are unique
    board=np.sort(board)
    return board


def possible_out (board):
    '''produce all possible outcomes (after 1 player) for a current board'''
    board=clean_board(board)
    num_unique, idx_unique=np.unique(board, return_index=1)
    out=[]
    for inum, iidx in zip(num_unique, idx_unique):
#         print (inum, iidx)
        splits=split_num (inum)
        for isplit in splits:
#             print ('board',board)
            iboard=list(copy.deepcopy(board))
            iboard=np.delete(iboard,iidx)
            iboard=np.concatenate([iboard,isplit]).astype(int)
#             print ('iboard',iboard)
            iboard=clean_board(iboard)
#             print ('inum,iboard',inum,iboard,type(iboard))
            if len(iboard)==1 and iboard[0]==1:
                return [[1]] ## if there's a board that can force others to lose, then stop here
            out.append(iboard)
    out = [list(x) for x in set(tuple(x) for x in out)] ## only count unique possibilities e.g. [1,2] and [1,2,1,1] would be the same after clean up
    return out

def status(board):
    # '''return True when reaching the leaf, or board clears []'''
    '''return True when there is [1] left, or lost'''
    # if len(board)==0:
    #     return True
    # elif len(board)==1 and board[0]>1:
    #     return True
    # else:
    #     return False
    if len(board)==0:
        return True
    elif len(board)==1 and board[0]==1:
        return True
    else:
        return False

board0=init_board(3)
print (board0)
boards=possible_out (board0)
print (boards)
for iboard in boards:
    print (iboard, status(iboard))
print ([1], status([1]))

[1 2 3]
[[1, 3], [1, 2], [2], [2, 3], [1, 2, 2], [3]]
[1, 3] False
[1, 2] False
[2] False
[2, 3] False
[1, 2, 2] False
[3] False
[1] True


In [116]:
import hashlib

def hash_list(lst):
    lst_bytes = str(lst).encode()  # Convert to string and encode
    return hashlib.sha256(lst_bytes).hexdigest()  # Generate SHA-256 hash

In [138]:
def build_stick_tree(board):
    """Builds a stick game tree where each node splits into all possible new boads"""
    root = TreeNode(board)

    def recursive_add(node):
        if status(node.value) and len(node.value)==0:
            return
        boards=possible_out (node.value)
        for iboard in boards:
            if len(iboard)==0:
                break
            ichild=TreeNode(iboard)
            node.add_child(ichild)
            recursive_add(ichild)

    recursive_add(root)
    return root


def build_stick_tree_cache (board, node_cache=None):
    """Builds a stick game tree where each node splits into all possible new boads"""

    # if len(board)==0:
    #     return
    if node_cache is None:
        node_cache = {}
    hash_board=hash_list(board)

    if hash_board in node_cache:
        return node_cache[hash_board]

    root = TreeNode(board)
    node_cache[hash_board] = root

    def recursive_add(node):
        if status(node.value) and len(node.value)==0:
            return
        boards=possible_out (node.value)
        for iboard in boards:
            if len(iboard)==0:
                break
            ihash_board=hash_list(iboard)
            if ihash_board in node_cache:
                ichild=node_cache[ihash_board]
            else:
                ichild=TreeNode(iboard)
                recursive_add(ichild)
                node_cache[ihash_board]=ichild

            node.add_child(ichild)

    recursive_add(root)
    return root

def print_tree(node, depth=0):
    """Prints the tree structure."""
    print("  " * depth + str(node.value))
    for child in node.children:
        print_tree(child, depth + 1)

# Example usage
lv=3  # Change this number to test different values
board=init_board(lv)
root = build_stick_tree(board)
print("Stick Tree:")
print_tree(root)

node_cache = {}
root_cache = build_stick_tree_cache (board, node_cache)
print("Stick Tree (Cache):")
print_tree(root_cache)

# def are_trees_identical(node1, node2):
#     """Checks if two trees are identical in structure and values."""
#     if node1 is None and node2 is None:
#         return True
#     if node1 is None or node2 is None:
#         return False
#     if node1.value != node2.value:
#         return False
#     if len(node1.children) != len(node2.children):
#         return False

#     return all(are_trees_identical(c1, c2) for c1, c2 in zip(node1.children, node2.children))

# print("Tree1 and Tree2 are identical:", are_trees_identical(root, root_cache))

Stick Tree:
[1 2 3]
  [1, 3]
    [1]
  [1, 2]
    [1]
  [2]
    [1]
  [2, 3]
    [1, 2]
      [1]
    [2]
      [1]
    [3]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
    [1, 3]
      [1]
  [1, 2, 2]
    [1, 2]
      [1]
    [2]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
  [3]
    [1]
Stick Tree (Cache):
[1 2 3]
  [1, 3]
    [1]
  [1, 2]
    [1]
  [2]
    [1]
  [2, 3]
    [1, 2]
      [1]
    [2]
      [1]
    [3]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
    [1, 3]
      [1]
  [1, 2, 2]
    [1, 2]
      [1]
    [2]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
  [3]
    [1]


In [149]:
%timeit build_stick_tree(board)
%timeit build_stick_tree_cache (board)

3.07 ms ± 657 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.08 ms ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.1 ms ± 51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [187]:
### now walk through the tree to find winning path
def find_leaf_levels_and_paths(node, level=0, path="", leaves=None):
    """Finds all leaf nodes, their levels, and paths in the tree."""
    if leaves is None:
        leaves = []

    if not node.children:  # If it's a leaf node
        leaves.append((node.value, level, path))

    for child in node.children:
        find_leaf_levels_and_paths(child, level + 1, path + f"->{child.value}", leaves)

    return leaves

# Example usage:

lv=3  # Change this number to test different values
board=init_board(lv)
root = build_stick_tree(board)
leaves = find_leaf_levels_and_paths(root)
print("Possible paths for stick game level", board)
for value, lvl, path in leaves:
    print(f"Leaf depth: {lvl} ({['LOST', 'WIN '][lvl%2]}), Path: {path}")

Possible paths for stick game level [1 2 3]
Leaf depth: 2 (LOST), Path: ->[1, 3]->[1]
Leaf depth: 2 (LOST), Path: ->[1, 2]->[1]
Leaf depth: 2 (LOST), Path: ->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[1, 2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[3]->[1]
Leaf depth: 4 (LOST), Path: ->[2, 3]->[2, 2]->[1, 2]->[1]
Leaf depth: 4 (LOST), Path: ->[2, 3]->[2, 2]->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[1, 3]->[1]
Leaf depth: 3 (WIN ), Path: ->[1, 2, 2]->[1, 2]->[1]
Leaf depth: 3 (WIN ), Path: ->[1, 2, 2]->[2]->[1]
Leaf depth: 4 (LOST), Path: ->[1, 2, 2]->[2, 2]->[1, 2]->[1]
Leaf depth: 4 (LOST), Path: ->[1, 2, 2]->[2, 2]->[2]->[1]
Leaf depth: 2 (LOST), Path: ->[3]->[1]


In [111]:
# ## use cache, from chatgpt
# def build_factorization_tree(n, node_cache=None):
#     """Builds a factorization tree with memoization."""
#     if node_cache is None:
#         node_cache = {}

#     # If this number was already processed, return the existing node
#     if n in node_cache:
#         return node_cache[n]

#     root = TreeNode(n)
#     node_cache[n] = root  # Store in cache before processing further

#     def recursive_factorize(node):
#         if is_prime(node.value):  # Stop when a prime is reached
#             return
#         factor1, factor2 = factorize(node.value)
#         if factor1 == node.value:  # If it's prime, stop
#             return

#         # Check cache before creating new nodes
#         if factor1 in node_cache:
#             child1 = node_cache[factor1]
#         else:
#             child1 = TreeNode(factor1)
#             node_cache[factor1] = child1

#         if factor2 in node_cache:
#             child2 = node_cache[factor2]
#         else:
#             child2 = TreeNode(factor2)
#             node_cache[factor2] = child2

#         node.add_child(child1)
#         node.add_child(child2)

#         # Recursively factorize only if not in cache
#         if factor1 not in node_cache:
#             recursive_factorize(child1)
#         if factor2 not in node_cache:
#             recursive_factorize(child2)

#     recursive_factorize(root)
#     return root

# def print_tree(node, depth=0):
#     """Prints the tree structure."""
#     print("  " * depth + str(node.value))
#     for child in node.children:
#         print_tree(child, depth + 1)

# # Example usage
# node_cache = {}  # Create a shared cache for multiple calls
# number = 120  # Change this number to test different values
# root = build_factorization_tree(number, node_cache)

# print("Factorization Tree:")
# print_tree(root)

# # Reuse cache for another number
# print("\nRecomputing for 120 should be instant:")
# root2 = build_factorization_tree(120, node_cache)  # Should reuse stored nodes
# print_tree(root2)

In [ ]:
## code block from chatgpt with prompt
## "write python code (can include numpy) to traverse a tree, including establish the tree"

# from collections import deque

# class TreeNode:
#     def __init__(self, value):
#         self.value = value
#         self.children = []

#     def add_child(self, child_node):
#         self.children.append(child_node)

# # Depth-First Search (DFS) Traversal

# def dfs_traversal(node, visited=None):
#     if visited is None:
#         visited = []

#     visited.append(node.value)
#     for child in node.children:
#         dfs_traversal(child, visited)

#     return visited

# # Breadth-First Search (BFS) Traversal

# def bfs_traversal(root):
#     visited = []
#     queue = deque([root])

#     while queue:
#         node = queue.popleft()
#         visited.append(node.value)
#         queue.extend(node.children)

#     return visited

# # Example: Constructing a tree
# root = TreeNode(1)
# child1 = TreeNode(2)
# child2 = TreeNode(3)
# child3 = TreeNode(4)
# child4 = TreeNode(5)

# root.add_child(child1)
# root.add_child(child2)
# child1.add_child(child3)
# child1.add_child(child4)

# # Traversing the tree
# print("DFS Traversal:", dfs_traversal(root))
# print("BFS Traversal:", bfs_traversal(root))

# def find_max_depth(node, depth=0):
#     """Finds the maximum depth of the tree."""
#     if not node.children:
#         return depth
#     return max(find_max_depth(child, depth + 1) for child in node.children)

# def get_last_level_nodes(root):
#     """Finds all nodes at the last level."""
#     max_depth = find_max_depth(root)
#     last_level_nodes = []

#     def collect_nodes_at_depth(node, depth):
#         if depth == max_depth:
#             last_level_nodes.append(node.value)
#         for child in node.children:
#             collect_nodes_at_depth(child, depth + 1)

#     collect_nodes_at_depth(root, 0)
#     return last_level_nodes

# # Example usage:
# print ('find max dept(node)', find_max_depth(root))
# print("Nodes at last level:", get_last_level_nodes(root))

In [7]:
# ## chatgpt "build a tree where a number is factorized into 2 numbers each time, until only prime numbers left"

# class TreeNode:
#     def __init__(self, value):
#         self.value = value
#         self.children = []

#     def add_child(self, child_node):
#         self.children.append(child_node)

# def is_prime(n):
#     """Check if a number is prime."""
#     if n < 2:
#         return False
#     for i in range(2, int(np.sqrt(n)) + 1):
#         if n % i == 0:
#             return False
#     return True

# def factorize(n):
#     """Finds the smallest factor (other than 1) of n."""
#     for i in range(2, int(np.sqrt(n)) + 1):
#         if n % i == 0:
#             return i, n // i
#     return n, 1  # If prime, return itself

# def build_factorization_tree(n):
#     """Builds a factorization tree where each node splits into two factors."""
#     root = TreeNode(n)

#     def recursive_factorize(node):
#         if is_prime(node.value):  # Stop when a prime is reached
#             return
#         factor1, factor2 = factorize(node.value)
#         if factor1 == node.value:  # If it's prime, stop
#             return
#         child1 = TreeNode(factor1)
#         child2 = TreeNode(factor2)
#         node.add_child(child1)
#         node.add_child(child2)
#         recursive_factorize(child1)
#         recursive_factorize(child2)

#     recursive_factorize(root)
#     return root

# def print_tree(node, depth=0):
#     """Prints the tree structure."""
#     print("  " * depth + str(node.value))
#     for child in node.children:
#         print_tree(child, depth + 1)

# # Example usage
# number = 120  # Change this number to test different values
# root = build_factorization_tree(number)

# print("Factorization Tree:")
# print_tree(root)
